In [65]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

In [66]:
def printmd(string):
    display(Markdown(string))

При чтении будем хранить данные в переменной `CACHE`, чтобы ускорить загрузку. В случае их обновления, достаточно вызвать `read(override=True)`

In [67]:
SMALL_DATASET_DIR = '../../nmnist/notMNIST_small/'
LARGE_DATASET_DIR = '../../nmnist/notMNIST_large/'
CACHE = {}
LABEL_MAP = {}
INV_LABEL_MAP = {}

In [68]:
def read(data_dir, override=False):
    f_v = 0
    global CACHE
    if not CACHE.get(data_dir, []) or override:
        CACHE[data_dir] = []
        X, y = [], []
        for f in tqdm(os.listdir(data_dir), desc='Letter'):
            if not f.startswith('.'):
                img_dir = os.path.join(data_dir, f)
                for img in os.listdir(img_dir):
                    img_path = os.path.join(img_dir, img)
                    data = cv2.imread(img_path, 0)
                    if data is None:
                        continue
                    X.append(data * 2 / 255 - 1)
                    if LABEL_MAP.get(f) is None:
                        LABEL_MAP[f] = f_v
                        INV_LABEL_MAP[f_v] = f
                        f_v += 1
                    y.append(LABEL_MAP[f])
        CACHE[data_dir].append(np.array(X))
        CACHE[data_dir].append(np.array(y))
    return CACHE[data_dir][0], CACHE[data_dir][1]

In [69]:
def get_data(data_dir, verbose=False, override=False):
    X, y = read(data_dir, override=override)
    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    if verbose:
        print(X.shape)
        print(y.shape)
        print(X[:5])
        print(y[:5])
        print(np.unique(y))
    return X, y

In [70]:
def get_split_data(data_dir, size=(0.7, 0.3), verbose=False, random_state=6, override=False):
    X, y = shuffle(*get_data(data_dir, verbose=verbose, override=override), random_state=random_state)
    assert abs(np.sum(size) - 1.0) < 0.001
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size[1], random_state=random_state)
    return X_train, y_train, X_test, y_test

In [71]:
X_train, y_train, X_test, y_test = get_split_data(SMALL_DATASET_DIR, override=True)

Построим 5 слоев с функциями активации `ReLU` и `softmax` на последнем слое, коэффициент обучения $0.001$, $100$ эпох:

In [72]:
lr = 0.001
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.001
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 6s 479us/step - loss: 2.1814 - acc: 0.2827
Epoch 2/100
13106/13106 [==============================] - 3s 211us/step - loss: 1.8574 - acc: 0.5022
Epoch 3/100
13106/13106 [==============================] - 3s 203us/step - loss: 1.4682 - acc: 0.6814
Epoch 4/100
13106/13106 [==============================] - 3s 207us/step - loss: 1.1076 - acc: 0.7703
Epoch 5/100
13106/13106 [==============================] - 3s 222us/step - loss: 0.8836 - acc: 0.8093
Epoch 6/100
13106/13106 [==============================] - 3s 204us/step - loss: 0.7539 - acc: 0.8269
Epoch 7/100
13106/13106 [==============================] - 3s 202us/step - loss: 0.6738 - acc: 0.8401
Epoch 8/100
13106/13106 [==============================] - 3s 215us/step - loss: 0.6218 - acc: 0.8486
Epoch 9/100
13106/13106 [==============================] - 3s 215us/step - loss: 0.5839 - ac

13106/13106 [==============================] - 2s 187us/step - loss: 0.2563 - acc: 0.9301
Epoch 80/100
13106/13106 [==============================] - 2s 182us/step - loss: 0.2542 - acc: 0.9312
Epoch 81/100
13106/13106 [==============================] - 2s 189us/step - loss: 0.2520 - acc: 0.9299
Epoch 82/100
13106/13106 [==============================] - 2s 186us/step - loss: 0.2494 - acc: 0.9314
Epoch 83/100
13106/13106 [==============================] - 2s 182us/step - loss: 0.2481 - acc: 0.9302
Epoch 84/100
13106/13106 [==============================] - 3s 201us/step - loss: 0.2456 - acc: 0.9320
Epoch 85/100
13106/13106 [==============================] - 2s 188us/step - loss: 0.2441 - acc: 0.9321
Epoch 86/100
13106/13106 [==============================] - 2s 187us/step - loss: 0.2418 - acc: 0.9328
Epoch 87/100
13106/13106 [==============================] - 2s 188us/step - loss: 0.2402 - acc: 0.9334
Epoch 88/100
13106/13106 [==============================] - 2s 187us/step - loss: 0.23

Изменим коэффициент обучения на больший `lr = 0.1`:

In [74]:
lr = 0.1
epochs = 50
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.1
Epochs: 50

Epoch 1/50
13106/13106 [==============================] - 7s 500us/step - loss: 0.8130 - acc: 0.7437
Epoch 2/50
13106/13106 [==============================] - 3s 197us/step - loss: 0.4181 - acc: 0.8751
Epoch 3/50
13106/13106 [==============================] - 3s 220us/step - loss: 0.3578 - acc: 0.8879
Epoch 4/50
13106/13106 [==============================] - 3s 249us/step - loss: 0.3233 - acc: 0.9003
Epoch 5/50
13106/13106 [==============================] - 3s 232us/step - loss: 0.2836 - acc: 0.9113
Epoch 6/50
13106/13106 [==============================] - 3s 262us/step - loss: 0.2604 - acc: 0.9177
Epoch 7/50
13106/13106 [==============================] - 4s 303us/step - loss: 0.2366 - acc: 0.9259
Epoch 8/50
13106/13106 [==============================] - 3s 210us/step - loss: 0.2303 - acc: 0.9263
Epoch 9/50
13106/13106 [==============================] - 3s 213us/step - loss: 0.2094 - acc: 0.9295
Ep

Точность увеличилась, попробуем найти оптимальный коэффициент обучения:

In [76]:
lr = 0.2
epochs = 50
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.2
Epochs: 50

Epoch 1/50
13106/13106 [==============================] - 5s 374us/step - loss: 1.0957 - acc: 0.6366
Epoch 2/50
13106/13106 [==============================] - 4s 273us/step - loss: 0.5060 - acc: 0.8438
Epoch 3/50
13106/13106 [==============================] - 4s 288us/step - loss: 0.3948 - acc: 0.8808
Epoch 4/50
13106/13106 [==============================] - 2s 188us/step - loss: 0.3563 - acc: 0.8901
Epoch 5/50
13106/13106 [==============================] - 2s 185us/step - loss: 0.3343 - acc: 0.8956
Epoch 6/50
13106/13106 [==============================] - 2s 183us/step - loss: 0.3060 - acc: 0.9049
Epoch 7/50
13106/13106 [==============================] - 3s 234us/step - loss: 0.2793 - acc: 0.9126
Epoch 8/50
13106/13106 [==============================] - 3s 194us/step - loss: 0.2602 - acc: 0.9161
Epoch 9/50
13106/13106 [==============================] - 3s 205us/step - loss: 0.2531 - acc: 0.9191
Ep

Как видно, точность слегка упала. Поэтому экспериментально будем считать, что коэффициент обучения, увеличение которого ухудшает точность, равен `lr_opt = 0.1`. 

Далее попробуем применить другие функции активации, например `tanh`:

In [78]:
lr = 0.1
epochs = 50
printmd('**Building with new architecture...**')
print('512(tanh) - 128(tanh) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dense(128, activation=tf.nn.tanh),
    keras.layers.Dense(32, activation=tf.nn.tanh),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(tanh) - 128(tanh) - 32(tanh) - 10(softmax)
Learning rate: 0.1
Epochs: 50

Epoch 1/50
13106/13106 [==============================] - 5s 394us/step - loss: 0.6870 - acc: 0.7992
Epoch 2/50
13106/13106 [==============================] - 3s 208us/step - loss: 0.4098 - acc: 0.8787
Epoch 3/50
13106/13106 [==============================] - 3s 201us/step - loss: 0.3511 - acc: 0.8943
Epoch 4/50
13106/13106 [==============================] - 3s 219us/step - loss: 0.3126 - acc: 0.9070
Epoch 5/50
13106/13106 [==============================] - 3s 226us/step - loss: 0.2878 - acc: 0.9116
Epoch 6/50
13106/13106 [==============================] - 3s 259us/step - loss: 0.2575 - acc: 0.9213
Epoch 7/50
13106/13106 [==============================] - 3s 219us/step - loss: 0.2393 - acc: 0.9257
Epoch 8/50
13106/13106 [==============================] - 3s 220us/step - loss: 0.2225 - acc: 0.9298
Epoch 9/50
13106/13106 [==============================] - 3s 217us/step - loss: 0.1981 - acc: 0.9384
Epoch 10/50
1

В целом результаты не улучшились

Попробуем добавить dropout (используем небольшой коэффициент, как рекомендуется здесь: https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

```Generally, use a small dropout value of 20%-50% of neurons with 20% providing a good starting point. A probability too low has minimal effect and a value too high results in under-learning by the network.```) и L2 регуляризатор (задачи L1 в целом решаются дропаутом):

In [84]:
lr = 0.1
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation=tf.nn.tanh, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.1
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 15s 1ms/step - loss: 5.3543 - acc: 0.7443
Epoch 2/100
13106/13106 [==============================] - 5s 396us/step - loss: 1.7255 - acc: 0.8274
Epoch 3/100
13106/13106 [==============================] - 5s 404us/step - loss: 1.0529 - acc: 0.8278
Epoch 4/100
13106/13106 [==============================] - 5s 398us/step - loss: 0.9096 - acc: 0.8287
Epoch 5/100
13106/13106 [==============================] - 6s 486us/step - loss: 0.8900 - acc: 0.8315
Epoch 6/100
13106/13106 [==============================] - 6s 471us/step - loss: 0.9015 - acc: 0.8264
Epoch 7/100
13106/13106 [==============================] - 6s 462us/step - loss: 0.9301 - acc: 0.8176
Epoch 8/100
13106/13106 [==============================] - 5s 393us/step - loss: 0.9137 - acc: 0.8237
Epoch 9/100
13106/13106 [==============================] - 6s 443us/step - loss: 0.9232 - acc: 0.8237
Epoc

13106/13106 [==============================] - 5s 405us/step - loss: 0.9978 - acc: 0.7950
Epoch 80/100
13106/13106 [==============================] - 5s 403us/step - loss: 0.9354 - acc: 0.8121
Epoch 81/100
13106/13106 [==============================] - 5s 404us/step - loss: 0.9724 - acc: 0.7983
Epoch 82/100
13106/13106 [==============================] - 5s 403us/step - loss: 0.9574 - acc: 0.8059
Epoch 83/100
13106/13106 [==============================] - 5s 402us/step - loss: 1.0022 - acc: 0.7989
Epoch 84/100
13106/13106 [==============================] - 5s 404us/step - loss: 0.9917 - acc: 0.8019
Epoch 85/100
13106/13106 [==============================] - 5s 406us/step - loss: 1.0041 - acc: 0.7966
Epoch 86/100
13106/13106 [==============================] - 5s 403us/step - loss: 0.9859 - acc: 0.7978
Epoch 87/100
13106/13106 [==============================] - 5s 406us/step - loss: 0.9763 - acc: 0.8016
Epoch 88/100
13106/13106 [==============================] - 5s 415us/step - loss: 1.01

Без регуляризатора:

In [85]:
lr = 0.1
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation=tf.nn.tanh),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.1
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 7s 502us/step - loss: 0.8214 - acc: 0.7533
Epoch 2/100
13106/13106 [==============================] - 3s 255us/step - loss: 0.5010 - acc: 0.8535
Epoch 3/100
13106/13106 [==============================] - 3s 253us/step - loss: 0.4362 - acc: 0.8668
Epoch 4/100
13106/13106 [==============================] - 3s 252us/step - loss: 0.4007 - acc: 0.8796
Epoch 5/100
13106/13106 [==============================] - 3s 252us/step - loss: 0.3711 - acc: 0.8884
Epoch 6/100
13106/13106 [==============================] - 3s 257us/step - loss: 0.3521 - acc: 0.8920
Epoch 7/100
13106/13106 [==============================] - 3s 253us/step - loss: 0.3368 - acc: 0.8978
Epoch 8/100
13106/13106 [==============================] - 3s 256us/step - loss: 0.3156 - acc: 0.9018
Epoch 9/100
13106/13106 [==============================] - 3s 264us/step - loss: 0.3123 - acc: 0.9040
Epo

13106/13106 [==============================] - 3s 255us/step - loss: 0.0713 - acc: 0.9769
Epoch 80/100
13106/13106 [==============================] - 3s 254us/step - loss: 0.0788 - acc: 0.9741
Epoch 81/100
13106/13106 [==============================] - 3s 255us/step - loss: 0.0800 - acc: 0.9741
Epoch 82/100
13106/13106 [==============================] - 3s 255us/step - loss: 0.0786 - acc: 0.9733
Epoch 83/100
13106/13106 [==============================] - 4s 268us/step - loss: 0.0773 - acc: 0.9742
Epoch 84/100
13106/13106 [==============================] - 3s 257us/step - loss: 0.0718 - acc: 0.9766
Epoch 85/100
13106/13106 [==============================] - 4s 296us/step - loss: 0.0712 - acc: 0.9771
Epoch 86/100
13106/13106 [==============================] - 3s 265us/step - loss: 0.0781 - acc: 0.9731
Epoch 87/100
13106/13106 [==============================] - 3s 257us/step - loss: 0.0729 - acc: 0.9759
Epoch 88/100
13106/13106 [==============================] - 3s 258us/step - loss: 0.07

Как видно, регуляризатор лишь ухудшил результат.

Попробуем применить динамический (адаптивный) коэффициент обучения. Для этого воспользуемся методом `Adam`:

In [80]:
lr = 0.001
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
loss_method = 'sparse_categorical_crossentropy'
model.compile(optimizer=tf.train.AdamOptimizer(lr), 
              loss=loss_method,
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.001
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 10s 749us/step - loss: 0.8558 - acc: 0.7429
Epoch 2/100
13106/13106 [==============================] - 7s 536us/step - loss: 0.5364 - acc: 0.8405
Epoch 3/100
13106/13106 [==============================] - 7s 514us/step - loss: 0.4724 - acc: 0.8618
Epoch 4/100
13106/13106 [==============================] - 7s 508us/step - loss: 0.4617 - acc: 0.8620
Epoch 5/100
13106/13106 [==============================] - 7s 505us/step - loss: 0.4279 - acc: 0.8694
Epoch 6/100
13106/13106 [==============================] - 7s 515us/step - loss: 0.4076 - acc: 0.8763
Epoch 7/100
13106/13106 [==============================] - 7s 510us/step - loss: 0.4032 - acc: 0.8761
Epoch 8/100
13106/13106 [==============================] - 7s 501us/step - loss: 0.3940 - acc: 0.8801
Epoch 9/100
13106/13106 [==============================] - 7s 518us/step - loss: 0.3720 - acc: 0.8872


13106/13106 [==============================] - 6s 434us/step - loss: 0.2133 - acc: 0.9303
Epoch 80/100
13106/13106 [==============================] - 6s 439us/step - loss: 0.2110 - acc: 0.9311
Epoch 81/100
13106/13106 [==============================] - 6s 435us/step - loss: 0.2158 - acc: 0.9307
Epoch 82/100
13106/13106 [==============================] - 6s 435us/step - loss: 0.2186 - acc: 0.9284
Epoch 83/100
13106/13106 [==============================] - 7s 508us/step - loss: 0.2128 - acc: 0.9299
Epoch 84/100
13106/13106 [==============================] - 6s 436us/step - loss: 0.2102 - acc: 0.9316
Epoch 85/100
13106/13106 [==============================] - 6s 435us/step - loss: 0.2078 - acc: 0.9329
Epoch 86/100
13106/13106 [==============================] - 6s 435us/step - loss: 0.2143 - acc: 0.9306
Epoch 87/100
13106/13106 [==============================] - 6s 432us/step - loss: 0.2033 - acc: 0.9345
Epoch 88/100
13106/13106 [==============================] - 6s 442us/step - loss: 0.21

In [82]:
lr = 0.01
epochs = 100
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
loss_method = 'sparse_categorical_crossentropy'
model.compile(optimizer=tf.train.AdamOptimizer(lr, beta1=0.8, beta2=0.9), 
              loss=loss_method,
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.01
Epochs: 100

Epoch 1/100
13106/13106 [==============================] - 9s 711us/step - loss: 2.3248 - acc: 0.1008
Epoch 2/100
13106/13106 [==============================] - 8s 596us/step - loss: 2.3068 - acc: 0.0989
Epoch 3/100
13106/13106 [==============================] - 6s 473us/step - loss: 2.3044 - acc: 0.0942
Epoch 4/100
13106/13106 [==============================] - 6s 478us/step - loss: 2.3041 - acc: 0.0970
Epoch 5/100
13106/13106 [==============================] - 6s 477us/step - loss: 2.3053 - acc: 0.0968
Epoch 6/100
13106/13106 [==============================] - 7s 497us/step - loss: 2.3054 - acc: 0.0979
Epoch 7/100
 7008/13106 [===============>..............] - ETA: 3s - loss: 2.3038 - acc: 0.1059

KeyboardInterrupt: 

<div style="text-align: right"> @chiselko6 </div>